參考:https://ithelp.ithome.com.tw/articles/10194403

In [1]:
from __future__ import print_function
import tensorflow as tf
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np
import json

In [2]:
count = 0
train_elements  = []
batch_size = 6  # Batch size for training.
epochs = 10  # Number of epochs to train for.
samples  = 2000
data_path = 'translation2019zh_train.json'
latent_dim = 256  # Latent dimensionality of the encoding space.

lines = open(data_path, encoding = 'utf-8')
for line in lines:
    count = count + 1
    if count < samples:
        train_elements .append(json.loads(line))
    else:
        break
lines.close()


In [3]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

for element in train_elements:
    entmp = element['english']
    chtmp = '\t' +element['chinese']
    input_texts.append(entmp)
    target_texts.append(chtmp)
    for char in entmp:
        if char not in input_characters:
            input_characters.add(char)
    for char in chtmp:
        if char not in target_characters:
            target_characters.add(char)

In [4]:
print(input_texts[0])
print(target_texts[0])

For greater sharpness, but with a slight increase in graininess, you can use a 1:1 dilution of this developer.
	为了更好的锐度，但是附带的会多一些颗粒度，可以使用这个显影剂的1：1稀释液。


In [5]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

In [6]:
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [7]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 1999
Number of unique input tokens: 185
Number of unique output tokens: 2870
Max sequence length for inputs: 256
Max sequence length for outputs: 121


In [8]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

前面設置太大記憶體會不夠 Memoryerror

In [9]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [10]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

In [11]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

In [12]:
encoder_states = [state_h, state_c]

In [13]:
decoder_inputs = Input(shape=(None, num_decoder_tokens))

In [14]:
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [15]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [16]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

Epoch 1/10
267/267 [==============================] - 317s 1s/step - loss: 2.0155 - val_loss: 2.0484
Epoch 2/10
267/267 [==============================] - 301s 1s/step - loss: 2.0002 - val_loss: 2.0502
Epoch 3/10
267/267 [==============================] - 292s 1s/step - loss: 1.9943 - val_loss: 2.0567
Epoch 4/10
267/267 [==============================] - 284s 1s/step - loss: 1.9922 - val_loss: 2.0526
Epoch 5/10
267/267 [==============================] - 287s 1s/step - loss: 1.9911 - val_loss: 2.0580
Epoch 6/10
267/267 [==============================] - 285s 1s/step - loss: 1.9897 - val_loss: 2.0532
Epoch 7/10
267/267 [==============================] - 283s 1s/step - loss: 1.9871 - val_loss: 2.0550
Epoch 8/10
267/267 [==============================] - 281s 1s/step - loss: 1.9868 - val_loss: 2.0561
Epoch 9/10
267/267 [==============================] - 281s 1s/step - loss: 1.9858 - val_loss: 2.0572
Epoch 10/10
267/267 [==============================] - 280s 1s/step - loss: 1.9847 - val_lo

In [17]:
model.save('s2smodel.h5')

Inference

In [18]:
encoder_model = Model(encoder_inputs, encoder_states)

In [19]:
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

In [20]:
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)

In [21]:
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [22]:
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [23]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [24]:
for seq_index in range(5):
    # Take one sequence (part of the training test)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('----------------------------------------------------------------')
    print('Input sentence:', input_texts[seq_index])
    try:
        print('Decoded sentence:', decoded_sentence)
    except:
        print('Decoded sentence:', decoded_sentence.encode('ascii', 'replace'))

----------------------------------------------------------------
Input sentence: For greater sharpness, but with a slight increase in graininess, you can use a 1:1 dilution of this developer.
Decoded sentence: 在的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的
----------------------------------------------------------------
Input sentence: He calls the Green Book, his book of teachings, “the new gospel.
Decoded sentence: 在的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的
----------------------------------------------------------------
Input sentence: And the light breeze moves me to caress her long ear
Decoded sentence: 在的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的
----------------------------------------------------------------
Input sentence: They have the blood of m

----------------------------------------------------------------
Input sentence: Fuler is one of 253 schools have credited by the Sociation of Phiological schools in the United States and Canada.
Decoded sentence: 在的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的
----------------------------------------------------------------
Input sentence: It shows that vertical stiffener's spaces have some effects on pure-shearing ulti…
Decoded sentence: 在的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的
----------------------------------------------------------------
Input sentence: "People are embarrassed to admit that's why they're giving up their pets, " said Betsy McFarland, the Humane Society's director of communications for companion animals.
Decoded sentence: 在的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的

----------------------------------------------------------------
Input sentence: He had formerly been in business at Bristol, but failed in debt to a number of people, compounded, and went to America.
Decoded sentence: 在的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的
----------------------------------------------------------------
Input sentence: Basing on the on site tests of anchor, authors found that anchors have obvious pre stress loss problem during stretching and locking, analyzed and proposed several solutions.
Decoded sentence: 在的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的
----------------------------------------------------------------
Input sentence: From hair tip first began gradually, after all, through from downward, nodular comb.
Decoded sentence: 在的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的

----------------------------------------------------------------
Input sentence: The idea of flipping from one entry to another, following a line of inquiry (especially etymological inquiry) from one page to another, even one volume to another, is a sensual experience.
Decoded sentence: 在的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的
----------------------------------------------------------------
Input sentence: Further Practice for Pairs ·Add a third speaker and create your own lines.
Decoded sentence: 在的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的
----------------------------------------------------------------
Input sentence: Still, Brasier asserts that the light carbon enrichments may well be able to form through lifeless chemical reactions—much as Fedo and others have argued could have occurred at Akilia.
Decoded sentence: 在的的的的的的的

----------------------------------------------------------------
Input sentence: Ran Hua (1961 ~), female, associate professor, PhD. candidate , School of Journalism & Communication, Wuhan University, majoring in communication theories.
Decoded sentence: 在的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的
----------------------------------------------------------------
Input sentence: Others include shrouding Earth in sun-reflecting aerosol particles, manufacturing CO2-absorbing artificial trees, and pumping CO2 into underground reservoirs.
Decoded sentence: 在的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的
